<a href="https://colab.research.google.com/github/ewvasconcelos/Preco_por_quilo/blob/main/Preco_por_quilo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 6.9 MB/s eta 0:00:00


In [2]:
#Importando as bibliotecas necessárias
from pathlib import Path
import hashlib
import google.generativeai as genai

from google.colab import userdata

In [3]:
#Configurando o modelo
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

In [18]:
system_instruction = "Você é um Especialista em comparação de preços de produtos por quilo."

jarbas = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

In [19]:
#Usando o modelo
chat = jarbas.start_chat(history=[])

while True:
  # Solicita os dados para o primeiro produto
  nome1 = input("Informe a descrição do PRODUTO 1: ")
  quantidade1 = input("Informe a quantidade em g/ml do PRODUTO 1: ")
  preco1 = float(input("Informe o preço do PRODUTO 1: "))

  # Solicita os dados para o segundo produto
  nome2 = input("Informe a descrição do PRODUTO 2: ")
  quantidade2 = input("Informe a quantidade em g/ml do PRODUTO 2: ")
  preco2 = float(input("Informe o preço do PRODUTO 2: "))

  # Converte a quantidade para float e remove 'g' ou 'ml'
  quantidade1 = float(quantidade1.replace("g", "").replace("ml", ""))
  quantidade2 = float(quantidade2.replace("g", "").replace("ml", ""))

  # Calcula o preço por quilo
  preco_por_quilo1 = (preco1 * 1000) / quantidade1
  preco_por_quilo2 = (preco2 * 1000) / quantidade2

  # Compara os preços por quilo
  if preco_por_quilo1 < preco_por_quilo2:
    melhor_opcao = f"{nome1} é a melhor opção, com preço por quilo de R$ {preco_por_quilo1:.2f}"
  elif preco_por_quilo2 < preco_por_quilo1:
    melhor_opcao = f"{nome2} é a melhor opção, com preço por quilo de R$ {preco_por_quilo2:.2f}"
  else:
    melhor_opcao = "Os produtos têm o mesmo preço por quilo."

  # Monta o prompt para o modelo
  prompt = [f"Compare os produtos {nome1} e {nome2}. O {nome1} custa R$ {preco1} por {quantidade1} g/ml e o {nome2} custa R$ {preco2} por {quantidade2} g/ml. Qual é a melhor opção considerando o custo benefício? Responda informando apenas o preço por quilo de cada produto e qual o percentual economizado", melhor_opcao]
  response = chat.send_message(prompt)

  # Imprime o histórico de mensagens
  import textwrap
  from IPython.display import display
  from IPython.display import Markdown

  def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  for message in chat.history:
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
    print('-------------------------------------------')

  # Encerra o loop após a comparação

  break

Informe a descrição do PRODUTO 1: azeite 250
Informe a quantidade em g/ml do PRODUTO 1: 250
Informe o preço do PRODUTO 1: 28
Informe a descrição do PRODUTO 2: azeite 400
Informe a quantidade em g/ml do PRODUTO 2: 400
Informe o preço do PRODUTO 2: 38


> **user**: Compare os produtos azeite 250 e azeite 400. O azeite 250 custa R$ 28.0 por 250.0 g/ml e o azeite 400 custa R$ 38.0 por 400.0 g/ml. Qual é a melhor opção considerando o custo benefício? Responda informando apenas o preço por quilo de cada produto e qual o percentual economizado

-------------------------------------------


> **model**: Azeite 250: R$ 112.00 por kg
> Azeite 400: R$ 95.00 por kg
> 
> Você economiza 15.18% com o Azeite 400. 


-------------------------------------------
